In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment5.ipynb")

# DS 453 / 653: Programming Assignment 5

**Due date**: Thursday, Feburary 29 at 8pm on [Gradescope](https://www.gradescope.com/courses/710247).

_You must follow the Academic Code of Conduct and Collaboration Policy stated in the course syllabus at all times while working on this assignment._

This assignment contains 2 questions worth a total of 5 points. You must receive at least 4 points to pass the assignment.

To begin, please execute the code block below:

In [2]:
import otter
grader = otter.Notebook()

## Assignment Overview

The goal of this assignment is to learn about consensus protocols in the synchronous setting. Specifically, we will study the Gradecast and phase-king protocols.

To do so, the instructors have provided code that will act as the network to connect a collection of parties. We provide the code below. While you should read it in detail to make sure you understand its behavior, for now just execute the code block and we will show an example below.

To instantiate these classes, execute the Python code below.

In [3]:
## Execute, but DO NOT MODIFY this code block ##

import random
from enum import IntEnum

# The Value enum contains all possible values that an honest or faulty party can send:
# either a bit (0 or 1) or a special catch-all value called "Null" for anything else.
class Value(IntEnum):
    Zero = 0
    One  = 1
    Null = -999

# The Party class is a base class representing a single participant in a distributed protocol.
# This party can choose what messages to send, and can record the contents of messages received.
class Party:
    def __init__(self, id: int, nb_faulty_parties: int, nb_total_parties: int, input: Value):
        # Precondition: we guarantee that the party's id is between 1 and n
        self.id = id
        # In your code, you should assume that the party is honest
        # (we handle faulty parties separately in the SynchronousNetwork class)
        self.input = input
        self.is_faulty = False
        self.f = nb_faulty_parties
        self.n = nb_total_parties

    # send and receive proceeds in rounds
    # our SynchronousNetwork always completes one round before starting the next one
    def send(self, round_number: int, destination_party: int) -> Value:
        return

    def receive(self, round_number: int, sender_party: int, val: Value) -> None:
        # do an action based on what you receive from the sender party
        # but don't return anything
        return None

    # the output method is only run once at the end of the protocol
    # after all rounds of communication are complete
    def output(self):
        ## return what this party decides to output
        return

class SynchronousNetwork:
    def __init__(self, PartyType, nb_faulty_parties: int,
                 nb_total_parties: int, inputs: list, debug=False):
        # verify that f < n
        assert(nb_faulty_parties < nb_total_parties)
        assert(len(inputs) == nb_total_parties)
        self.debug = debug

        # creating several parties with the prescribed inputs
        self.parties = [PartyType(i, nb_faulty_parties, nb_total_parties,
                                  inputs[i-1]) for i in range(1, nb_total_parties + 1)]
        self.debug_print("Inputs:")
        for i in range(1, nb_total_parties + 1):
            self.debug_print("Party " + str(i) + " input: " + inputs[i-1].name)

        # randomly set some of the parties to be faulty
        for p in random.sample(self.parties, nb_faulty_parties):
            p.is_faulty = True
        self.leader = random.sample(self.parties, 1)[0]

    def run(self, nb_rounds) -> list:
        # execute all nb_rounds rounds of the synchronous protocol, in order
        for i in range(1, nb_rounds + 1):
            self.debug_print("\nStart of round " + str(i) + ":")

            for p1 in self.parties:
                for p2 in self.parties:
                    # each party can send one message to all parties (including itself!)
                    val = SynchronousNetwork.send_with_errors(i, p1, p2)
                    self.debug_print("party with id " + str(p1.id) + " sending to party with id "
                                         + str(p2.id) + " a message with content: " + val.name)

                    # recipient receives the message instantaneously
                    p2.receive(i, p1.id, val)

        # after all rounds are finished, retrieve each party's output
        res = [p.output() for p in self.parties]

        # the faulty party has no output
        for i in range(len(self.parties)):
            if(self.parties[i].is_faulty == True):
                res[i] = Value.Null

        self.debug_print("\nOutputs are saved as the return value")
        return res

    def send_with_errors(round_nb, sender, recv) -> Value:
        # for honest parties, call the sender party and perform the action it wants
        if (not sender.is_faulty):
            return sender.send(round_nb, recv)
        # in this homework, a faulty party chooses a value to send at random
        # note that independent randomness is used for each message sent, in each round
        else:
            return SynchronousNetwork.randomValue()

    def randomValue() -> Value:
        r = random.random()
        if(r < 0.4): return Value.Zero
        if(r < 0.8): return Value.One
        else:        return Value.Null

    # pretty-printer that you can use to view the network communication
    def debug_print(self, obj):
        if self.debug:
            print(obj)
        else:
            pass

This code block provides two classes: a `Party` object that represents a single computer in a network, and a `SynchronousNetwork` that allows several `Party` objects to communicate.

Within each message communication, the sender can pick a `Value` from three options: `Value.Zero`, `Value.One`, or `Value.Null`. In words, the sender can choose any bit value (0 or 1), or it can send nothing.

The overall network contains `nb_total_parties` parties, of which a subset of `nb_faulty_parties` are faulty. Faulty parties behave chaotically: they send random messages and they never produce an output.

The network runs all of the parties (roughly) in parallel. You only have to write the code from the perspective of a single party $i$, and the network will handle the execution of all parties.

__An example.__ To give you an example of what we mean by this, consider a 1-round protocol in which all odd-numbered parties send `Value.One` and the even-numbered parties send `Value.Zero`. Here is how we can write the code that explains how a single party behaves.

In [4]:
class ParityParty(Party):
    def __init__(self, *args, **kwargs):  # has the same arguments as the base class
        super().__init__(*args, **kwargs) # executes init from the base class

    def send(self, round_number: int, destination_party: int) -> Value:
        if(self.id % 2 == 1):
            return Value.One
        else:
            return Value.Zero

    def receive(self, round_number: int, sender_party: int, val: Value) -> None:
        return
    
    def output(self):
        # in this example we always return a value of 0, regardless of inputs or messages
        return Value.Zero

Now let's connect four of these parity parties together. The neat thing is that even though we wrote the protocol from the 'local' perspective of a single party, the `SynchronousNetwork` class will now create the 'global' view of all parties in the network.

We will also give each party an input value, even though they don't use it in this simple protocol. Let's see what happens if we run this network, where one of the parties is faulty.

In [5]:
parity_inputs = [Value.One, Value.Zero, Value.One, Value.Zero]
parityNetwork = SynchronousNetwork(ParityParty, 1, 4, parity_inputs, debug=True)
result = parityNetwork.run(1) # the number 1 tells the network to run for one round
print(result)

Inputs:
Party 1 input: One
Party 2 input: Zero
Party 3 input: One
Party 4 input: Zero

Start of round 1:
party with id 1 sending to party with id 1 a message with content: One
party with id 1 sending to party with id 2 a message with content: One
party with id 1 sending to party with id 3 a message with content: One
party with id 1 sending to party with id 4 a message with content: One
party with id 2 sending to party with id 1 a message with content: One
party with id 2 sending to party with id 2 a message with content: Null
party with id 2 sending to party with id 3 a message with content: Null
party with id 2 sending to party with id 4 a message with content: One
party with id 3 sending to party with id 1 a message with content: One
party with id 3 sending to party with id 2 a message with content: One
party with id 3 sending to party with id 3 a message with content: One
party with id 3 sending to party with id 4 a message with content: One
party with id 4 sending to party with id 

As you can see, the first 3 parties followed the protocol. They sent a consistent value to each party in the system... including a message to themselves!

But party 4 is faulty. It disregards the rules of the game: rather than sending a `Value.Zero` to everyone, it has sent `Value.One` to one party, `Value.Zero` to two parties, and `Value.Null` to one party. Also, the faulty party has no output -- more precisely, it always outputs `Value.Null`.

Hopefully this gives you a flavor of how the `SynchronousNetwork` class connects the parties together. Now let's try building a real protocol in this way.

### Question 1: Construct the Gradecast protocol

__Your task__: In the code block below, write Python code that executes one party's component of the Gradecast protocol. Additionally, comment your code to describe each part of the protocol in words.

__Your response:__

In [6]:
class GradecastParty(Party):
    def __init__(self, *args, **kwargs):  # has the same arguments as the base class
        super().__init__(*args, **kwargs) # executes init from the base class
        
        # *** #
        self.values_received = {1: [], 2: []}

        # description: added a dictionary to keep track of the number of times a value is received
        # *** #


    def send(self, round_number: int, destination_party: int):

        # *** #
        # description: 
        # if it's the first round, send the party's input
        # if it's the second round, send the value that was received the most
        # otherwise, send Null
        # *** #

        if round_number == 1:
            return self.input
        elif round_number == 2:
            for value in set(self.values_received[1]):
                if self.values_received[1].count(value) >= self.n - self.f:
                    return value
        return Value.Null

    def receive(self, round_number: int, sender_party: int, val: Value):
        
        # *** #
        # description:
        # append value to the list of values received in the corresponding round
        # *** #

        self.values_received[round_number].append(val)

    def output(self):
        # TODO: complete this function
        zero_total = len([zero for zero in self.values_received[2] if zero == Value.Zero])
        one_total = len([one for one in self.values_received[2] if one == Value.One])

        if one_total >= self.n - self.f:
            return (Value.One, 2)
        elif zero_total >= self.n - self.f:
            return (Value.Zero, 2)
        elif one_total >= self.f + 1:
            return (Value.One, 1)
        elif zero_total >= self.f + 1:
            return (Value.Zero, 1)
        else:
            return (self.input, 0)

In [7]:
skewed_zeros = [Value.Zero, Value.Zero, Value.Zero, Value.One]
gradecastNetwork = SynchronousNetwork(GradecastParty, 1, 4, skewed_zeros, debug=True)
result = gradecastNetwork.run(2)

print(result)

Inputs:
Party 1 input: Zero
Party 2 input: Zero
Party 3 input: Zero
Party 4 input: One

Start of round 1:
party with id 1 sending to party with id 1 a message with content: Zero
party with id 1 sending to party with id 2 a message with content: Zero
party with id 1 sending to party with id 3 a message with content: Zero
party with id 1 sending to party with id 4 a message with content: Zero
party with id 2 sending to party with id 1 a message with content: Zero
party with id 2 sending to party with id 2 a message with content: Zero
party with id 2 sending to party with id 3 a message with content: Zero
party with id 2 sending to party with id 4 a message with content: Zero
party with id 3 sending to party with id 1 a message with content: Zero
party with id 3 sending to party with id 2 a message with content: Zero
party with id 3 sending to party with id 3 a message with content: Zero
party with id 3 sending to party with id 4 a message with content: Zero
party with id 4 sending to par

Once you are satisfied that your code works properly, you can run the tests below. There are two tests: one to check the validity property, and another to check the knowledge of agreement property. Each test performs many executions of the Gradecast 

_Points:_ 2

In [8]:
grader.check("q1")

q1 results: All test cases passed!

### Question 2: Construct the Phase-King protocol

__Your task:__ In the code block below, write Python code that executes one party's component of the phase-king protocol.

Once again, you may wish to review the lecture notes before starting this question. Make sure to describe your work using code comments and/or a written text explanation.

__Your response:__

_Points:_ 3

In [9]:
class PhaseKingParty(Party):
    def __init__(self, *args, **kwargs):  # has the same arguments as the base class
        super().__init__(*args, **kwargs) # executes init from the base class

        # create grade and terminate variables
        self.grade = 0
        self.gc = GradecastParty(self.id, self.f, self.n, self.input)
        self.terminate = [False] * self.n
        
    def send(self, round_number: int, destination_party: int) -> Value:

        # if it's the first round and the party is the leader, send the party's input
        if round_number % 3 == 1 and round_number // 3 + 1 == self.id:

            v, g = self.gc.output()
            self.input = v
            self.grade = g

            self.gc = GradecastParty(self.id, self.f, self.n, self.input)
            
            return self.input

        # if it's the second round, send the value that was received the most
        elif round_number % 3 == 2:
            return self.gc.send(1, destination_party)
        
        # if it's the third round, send the value that was received the most
        elif round_number % 3 == 0:
            return self.gc.send(2, destination_party)
        
        return Value.Null
            
    def receive(self, round_number: int, sender_party: int, val: Value) -> None:

        # if it's the first round, receive the value and grade if self.grade < 2
        if round_number % 3 == 1 and round_number // 3 + 1 == sender_party:
            
            v, g = self.gc.output()
            self.input = v
            self.grade = g
            
            if self.grade < 2:
                self.input = val            
            
            self.gc = GradecastParty(self.id, self.f, self.n, self.input)
            return 
        
        # if it's the second round, receive the value
        elif round_number % 3 == 2:
            return self.gc.receive(1, sender_party, val)
        
        # if it's the third round, receive the value
        elif round_number % 3 == 0:
            self.gc.receive(2, sender_party, val) 
            return
 
    def output(self):            
        return self.input

In [10]:
def test2():
    consistent_zeros = [Value.Zero, Value.Zero, Value.Zero, Value.Zero]
    skewed_zeros = [Value.Zero, Value.Zero, Value.Zero, Value.One]
    balanced_inputs = [Value.Zero, Value.Zero, Value.One, Value.One]
    skewed_ones = [Value.Zero, Value.One, Value.One, Value.One]
    consistent_ones = [Value.One, Value.One, Value.One, Value.One]
    for input in [consistent_zeros, skewed_zeros, balanced_inputs, skewed_ones, consistent_ones]:
        for _ in range(8000):
            phaseking = SynchronousNetwork(PhaseKingParty, 1, 4, input, debug=False)
            result = phaseking.run(6)
            if phaseking.parties[0].is_faulty == False:
                expected_output = input[0]
            else:
                expected_output = result[1]
            for i in range(4):
                if phaseking.parties[i].is_faulty == False:
                    assert result[i] == expected_output
    return True

test2()

True

In [11]:
grader.check("q2")

q2 results: All test cases passed!

## Submitting the Assignment

Please follow these instructions to complete the assignment and submit it for credit.

**Documenting collaborators, sources, and AI tools:** In accordance with the collaboration policy, use the space below to report if you used any resources to complete this homework assignment, aside from the lecture notes and the course textbooks/videos. Specifically, please report:

1. Names of all classmates you worked with, and a short description of the work that you performed together.
2. All written materials that you used, such as books or websites (besides the lecture notes or textbooks). Please include links to any web-based resources, or citations to any physical works.
3. All code that you used from other sources. In particular, if you used an AI tool, then you must include the entire exchange with the AI tool, as per the [CDS Generative AI Assistance Policy](https://www.bu.edu/cds-faculty/culture-community/gaia-policy/).

Remember that if we discover any undocumented collaborators, sources, or AI tools then this is grounds for a grade penalty and referral to BU's Academic Conduct Committee (as described in the syllabus).

_Your response:_

1. Anush

2.

3.

**Sending to Gradescope:** After completing the assignment:
- if you did the assignment on Colab, download it in `.ipynb` format.
- if you did the assignment locally on your machine, all you need to do is to find it in your directory.

Then, submit only the `.ipynb` file to this week's programming assignment on Gradescope. It may take a few seconds or a minute for the auto-grading system to check your work.

Remember that you can submit as many times as you want until the deadline for the assignment; only your last score counts.